(numpyro_conversion)=
# Converting NumPyro objects to DataTree

{class}`~datatree.DataTree` is the data format ArviZ relies on.

This page covers multiple ways to generate a `DataTree` from NumPyro MCMC and SVI objects.

```{seealso}

- Conversion from Python, numpy or pandas objects
- {ref}`xarray_for_arviz` for an overview of `InferenceData` and its role within ArviZ.
- {ref}`schema` describes the structure of `InferenceData` objects and the assumptions made by ArviZ to ease your exploratory analysis of Bayesian models.
```

We will start by importing the required packages and defining the model. The famous 8 school model.

In [ ]:
import arviz_base as az
import numpy as np

import numpyro
import numpyro.distributions as dist
from numpyro.infer import MCMC, NUTS, SVI, Trace_ELBO, autoguide
from jax import random
import jax.numpy as jnp

In [ ]:
J = 8
y_obs = np.array([28.0, 8.0, -3.0, 7.0, -1.0, 1.0, 18.0, 12.0])
sigma = np.array([15.0, 10.0, 16.0, 11.0, 9.0, 11.0, 10.0, 18.0])

In [ ]:
def eight_schools_model(J, sigma, y=None):
    mu = numpyro.sample("mu", dist.Normal(0, 5))
    tau = numpyro.sample("tau", dist.HalfCauchy(5))
    with numpyro.plate("J", J):
        eta = numpyro.sample("eta", dist.Normal(0, 1))
        theta = numpyro.deterministic("theta", mu + tau * eta)
        return numpyro.sample("obs", dist.Normal(theta, sigma), obs=y)
    

def eight_schools_custom_guide(J, sigma, y=None):

    # Variational parameters for mu
    mu_loc = numpyro.param("mu_loc", 0.0)
    mu_scale = numpyro.param("mu_scale", 1.0, constraint=dist.constraints.positive)
    mu = numpyro.sample("mu", dist.Normal(mu_loc, mu_scale))

    # Variational parameters for tau (positive support)
    tau_loc = numpyro.param("tau_loc", 1.0)
    tau_scale = numpyro.param("tau_scale", 0.5, constraint=dist.constraints.positive)
    tau = numpyro.sample("tau", dist.LogNormal(jnp.log(tau_loc), tau_scale))

    # Variational parameters for eta
    eta_loc = numpyro.param("eta_loc", jnp.zeros(J))
    eta_scale = numpyro.param("eta_scale", jnp.ones(J), constraint=dist.constraints.positive)
    with numpyro.plate("J", J):
        eta = numpyro.sample("eta", dist.Normal(eta_loc, eta_scale))

        # Deterministic transform
        numpyro.deterministic("theta", mu + tau * eta)

## Convert from MCMC

This first example shows conversion from MCMC

In [ ]:
# fit with MCMC
nuts = NUTS(eight_schools_model)
mcmc = MCMC(nuts, num_warmup = 1000, num_samples = 1000, num_chains=4)
mcmc.run(random.PRNGKey(0), J=J, sigma=sigma, y=y_obs, extra_fields=("num_steps", "energy"),)

# Convert to MCMC
idata_mcmc = az.from_numpyro(mcmc)
idata_mcmc

## Convert from SVI with Autoguide

In [ ]:
eight_schools_guide = autoguide.AutoNormal(eight_schools_model, init_loc_fn=numpyro.infer.init_to_median(num_samples=100))
svi = SVI(
    eight_schools_model, 
    guide=eight_schools_guide,
    optim=numpyro.optim.Adam(0.01),
    loss = Trace_ELBO()
)
svi_result = svi.run(random.PRNGKey(0), num_steps=10000, J=J, sigma=sigma, y=y_obs)
idata_svi = az.from_numpyro_svi(
    svi,
    svi_result=svi_result,
    model_kwargs=dict(J=J, sigma=sigma, y=y_obs), # SVI requires providing the fit args/kwargs
    num_samples = 4000 # number of samples to draw in the posterior
)
idata_svi

## Converting from SVI with a custom guide function

In [ ]:
svi_custom_guide = SVI(
    eight_schools_model, 
    guide=eight_schools_custom_guide,
    optim=numpyro.optim.Adam(0.01),
    loss = Trace_ELBO()
)
svi_custom_guide_result = svi_custom_guide.run(random.PRNGKey(0), num_steps=10000, J=J, sigma=sigma, y=y_obs)

idata_svi_custom_guide = az.from_numpyro_svi(
    svi_custom_guide,
    svi_result=svi_custom_guide_result,
    model_kwargs=dict(J=J, sigma=sigma, y=y_obs), # SVI requires providing the fit args/kwargs
    num_samples = 4000 # number of samples to draw in the posterior
)
idata_svi_custom_guide

## 

(emcee_conversion)=
# Converting emcee objects to DataTree

{class}`~datatree.DataTree` is the data format ArviZ relies on.

This page covers multiple ways to generate a `DataTree` from emcee objects.

In [ ]:
def log_prior_8school(theta):
    mu, tau, eta = theta[0], theta[1], theta[2:]
    # Half-cauchy prior, hwhm=25
    if tau < 0:
        return -np.inf
    prior_tau = -np.log(tau**2 + 25**2)
    prior_mu = -((mu / 10) ** 2)  # normal prior, loc=0, scale=10
    prior_eta = -np.sum(eta**2)  # normal prior, loc=0, scale=1
    return prior_mu + prior_tau + prior_eta


def log_likelihood_8school(theta, y, s):
    mu, tau, eta = theta[0], theta[1], theta[2:]
    return -(((mu + tau * eta - y) / s) ** 2)


def lnprob_8school(theta, y, s):
    prior = log_prior_8school(theta)
    like_vect = log_likelihood_8school(theta, y, s)
    like = np.sum(like_vect)
    return like + prior

In [ ]:
nwalkers = 40  # called chains in ArviZ
ndim = J + 2
draws = 1500
pos = np.random.normal(size=(nwalkers, ndim))
pos[:, 1] = np.absolute(pos[:, 1])
sampler = emcee.EnsembleSampler(nwalkers, ndim, lnprob_8school, args=(y_obs, sigma))
sampler.run_mcmc(pos, draws);

## Manually set variable names
This first example will show how to convert manually setting the variable names only, leaving everything else to ArviZ defaults.

In [ ]:
# define variable names, it cannot be inferred from emcee
var_names = ["mu", "tau"] + ["eta{}".format(i) for i in range(J)]
idata1 = az.from_emcee(sampler, var_names=var_names)
idata1

ArviZ has stored the posterior variables with the provided names as expected, but it has also included other useful information in the `InferenceData` object. The log probability of each sample is stored in the `sample_stats` group under the name `lp` and all the arguments passed to the sampler as `args` have been saved in the `observed_data` group.

It can also be useful to perform a burn in cut to the MCMC samples (see {meth}`arviz.InferenceData.sel` for more details)

In [ ]:
#idata1.sel(draw=slice(100, None))

From an InferenceData object, ArviZ's native data structure, the {func}`posterior plot <arviz.plot_posterior>` of a few variables can be done in one line:

In [ ]:
#az.plot_posterior(idata1, var_names=["mu", "tau", "eta4"])

## Structuring the posterior as multidimensional variables

This way of calling ``from_emcee`` stores each `eta` as a different variable, called `eta#`, 
however, they are in fact different dimensions of the same variable. 
This can be seen in the code of the likelihood and prior functions, where `theta` is unpacked as:

    mu, tau, eta = theta[0], theta[1], theta[2:]
    
ArviZ has support for multidimensional variables, and there is a way to tell it how to split the variables like it was done in the likelihood and prior functions:

In [ ]:
idata2 = az.from_emcee(sampler, slices=[0, 1, slice(2, None)])
idata2

After checking the default variable names, the trace of one dimension of eta can be plotted using ArviZ syntax:

In [ ]:
#az.plot_trace(idata2, var_names=["var_2"], coords={"var_2_dim_0": 4});

## `blobs`: unlock sample stats, posterior predictive and miscellanea

Emcee does not store per-draw sample stats, however, it has a functionality called
blobs that allows to store any variable on a per-draw basis. It can be used
to store some sample_stats or even posterior_predictive data. 

You can modify the probability function to use this ``blobs`` functionality and store the pointwise log likelihood,
then rerun the sampler using the new function:

In [ ]:
def lnprob_8school_blobs(theta, y, s):
    prior = log_prior_8school(theta)
    like_vect = log_likelihood_8school(theta, y, s)
    like = np.sum(like_vect)
    return like + prior, like_vect


sampler_blobs = emcee.EnsembleSampler(
    nwalkers,
    ndim,
    lnprob_8school_blobs,
    args=(y_obs, sigma),
)
sampler_blobs.run_mcmc(pos, draws);

You can now use the `blob_names` argument to indicate how to store this blob-defined variable. As the group is not specified, it will go to sample_stats.
Note that the argument blob_names is added to the arguments covered in the previous examples and we are also introducing `coords` and `dims` arguments to show the power and flexibility of the converter. For more on `coords` and `dims` see `page_in_construction`.

In [ ]:
dims = {"eta": ["school"], "log_likelihood": ["school"]}
idata3 = az.from_emcee(
    sampler_blobs,
    var_names=["mu", "tau", "eta"],
    slices=[0, 1, slice(2, None)],
    blob_names=["y"],
    dims=dims,
    coords={"school": range(8)},
)
idata3

## Multi-group blobs

You might even have more complicated blobs, each corresponding to a different group of the InferenceData object. Moreover, you can store the variables passed to the ``EnsembleSampler`` via the ``args`` argument in observed or constant data groups. This is shown in the example below:

In [ ]:
sampler_blobs.blobs[0, 1]

In [ ]:
def lnprob_8school_blobs(theta, y, sigma):
    mu, tau, eta = theta[0], theta[1], theta[2:]
    prior = log_prior_8school(theta)
    like_vect = log_likelihood_8school(theta, y, sigma)
    like = np.sum(like_vect)
    # store pointwise log likelihood, useful for model comparison with az.loo or az.waic
    # and posterior predictive samples as blobs
    return like + prior, (like_vect, np.random.normal((mu + tau * eta), sigma))


sampler_blobs = emcee.EnsembleSampler(
    nwalkers,
    ndim,
    lnprob_8school_blobs,
    args=(y_obs, sigma),
)
sampler_blobs.run_mcmc(pos, draws)

dims = {"eta": ["school"], "log_likelihood": ["school"], "y": ["school"]}
idata4 = az.from_emcee(
    sampler_blobs,
    var_names=["mu", "tau", "eta"],
    slices=[0, 1, slice(2, None)],
    arg_names=["y", "sigma"],
    arg_groups=["observed_data", "constant_data"],
    blob_names=["y", "y"],
    blob_groups=["log_likelihood", "posterior_predictive"],
    dims=dims,
    coords={"school": range(8)},
)
idata4

This last version, which contains both observed data and posterior predictive can be used to plot posterior predictive checks:

In [ ]:
#az.plot_ppc(idata4, var_names=["y"], alpha=0.3, num_pp_samples=200);

In [ ]:
%load_ext watermark
%watermark -n -u -v -iv -w